In [155]:
import os
import talib 
import numpy as np  
import pandas as pd 
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt 
# import mpl_finance as mpf
import matplotlib.dates as mdates
import mplfinance as mpf
import requests
import json
import datetime
from hs_udata import stock_quote_daily
from mpl_finance import candlestick_ohlc, candlestick2_ochl
import pandas_datareader as web
from datetime import datetime, timedelta
# from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.ticker as ticker
import warnings
import multiprocessing
from multiprocessing import Pool
warnings.filterwarnings('ignore')

In [ ]:

def print_klines_span(stock_code, cur_date, buy_idx, start_idx, dayn):
    if not os.path.exists(cur_date + '/png_profit'):
        os.makedirs(cur_date + '/png_profit')
    output_path = cur_date + '/png_profit/'+str(stock_code) + '_' + str(buy_idx) +'.png'
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    #获取macd
#     data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])

    #获取10日均线和30日均线
#     data["ma10"] = talib.MA(data["Close"], timeperiod=10)
#     data["ma30"] = talib.MA(data["Close"], timeperiod=30)

    #获取rsi
#     data["rsi"] = talib.RSI(data["Close"])
    data = data[start_idx:min(start_idx+dayn, data.index[-1])]
    data.index = range(0,len(data.index))
    
    fig = plt.figure()  #创建绘图区，包含四个子图
    fig.set_size_inches((20, 16))
    ax_candle = fig.add_axes((0, 0.72, 1, 0.32))   #蜡烛图子图
    ax_macd = fig.add_axes((0, 0.48, 1, 0.2), sharex=ax_candle)  #macd子图
    ax_rsi = fig.add_axes((0, 0.24, 1, 0.2), sharex=ax_candle)  #rsi子图
    ax_vol = fig.add_axes((0, 0, 1, 0.2), sharex=ax_candle)   #成交量子图
    ax_vol = fig.add_axes((0, 0, 1, 0.2))   #成交量子图

    ohlc = []   #存放行情数据，candlestick_ohlc需要传入固定格式的数据
    row_number = 0
    for date, row in data.iterrows():
        date, openp, closep, highp, lowp = row[:5]
        ohlc.append([row_number, openp, highp, lowp, closep])
        row_number = row_number+1

#     date_tickers = data.Date.values #获取Date数据
    date_tickers = ['-'.join(val.split('-')[1:]) for val in data.loc[data[data.index % 4 == 0].index].Date.values] #获取Date数据
    date_tickers = np.insert(date_tickers, 0 ,['0','0'])

    #绘制蜡烛图
    ax_candle.plot(data.index, data["ma10"], label="MA10")
    ax_candle.plot(data.index, data["ma30"], label="MA30")
    candlestick_ohlc(ax_candle, ohlc, colorup="r", colordown="g", width=0.8)
    ax_candle.xaxis.set_major_locator(ticker.MultipleLocator(4))
    ax_candle.xaxis.set_major_formatter(ticker.FixedFormatter(date_tickers)) #设置x轴刻度
    ax_candle.grid(True)
    ax_candle.set_title(str(stock_code), fontsize=20)
    ax_candle.legend()

    #绘制MACD
    ax_macd.plot(data.index, data["macd"], label="macd")
    ax_macd.bar(data.index, data["macd_hist"] * 3, label="hist")
    ax_macd.plot(data.index, data["macd_signal"], label="signal")
    ax_macd.set_title('MACD')
    ax_macd.legend()

    #绘制RSI
    ax_rsi.set_ylabel("(%)")
    ax_rsi.plot(data.index, [70] * len(data.index), label="overbought")
    ax_rsi.plot(data.index, [30] * len(data.index), label="oversold")
    ax_rsi.plot(data.index, data["rsi"], label="rsi")
    ax_rsi.set_title('KDJ')
    ax_rsi.legend()

    #绘制成交量
    ax_vol.bar(data.index, data["Volume"] / 1000000)
    ax_vol.set_ylabel("(Million)")

    plt.savefig(cur_date + '/png_profit/'+str(stock_code) + '_' + str(buy_idx) +'.png', bbox_inches="tight")

In [27]:
print_klines_span(603596, '20230109', 114, 64, 40)

NameError: name 'print_klines_span' is not defined

In [159]:
def if_macd_back(stock_code, date, time_span):
    data_dir = 'data'
    if time_span == 60:
        data_dir = 'data_60'
    elif time_span == 30:
        data_dir = 'data_30'
    file = os.path.join(date + '/' + data_dir, str(stock_code)+'.csv')
    data = pd.read_csv(file)
    indexs = data[data['macd_hist'] < 0].index
    end2 = indexs[len(indexs)-1]
    start2 = -1
    start1 = -1
    end1 = -1
#     print(indexs)
    idx = -1
    for i in range(len(indexs)-2, -1, -1):
#         print(indexs[i]+1, indexs[i+1])
        if indexs[i]+1 == indexs[i+1]:
            continue
        else:
            start2 =  indexs[i+1]
            end1 = indexs[i]
            idx = i
            break
#     print(idx)
    for i in range(idx-1,-1,-1):
#         print(indexs[i]+1, indexs[i+1])
        if indexs[i]+1 == indexs[i+1]:
            continue
        else:
            start1 =  indexs[i+1]
            break
    if start1 == -1 or start2 == -1 or end1 == -1:
        return False
    high1_min = data.loc[start1]['Close']
    high2_min = data.loc[start2]['Close']
    area1 = 0.0
    area2 = 0.0
    if end2 != len(indexs)-1:
        for i in range(start1, end1+1):
#             print(i, data.loc[i]['macd_hist'])
            area1 += data.loc[i]['macd_hist']
            if high1_min > data.loc[i]['Close']:
                high1_min = data.loc[i]['Close']
        for i in range(start2, end2+1):
#             print(i, data.loc[i]['macd_hist'])
            area2 += data.loc[i]['macd_hist']
            if high2_min > data.loc[i]['Close']:
                high2_min = data.loc[i]['Close']
    
#     print(start1, end1, start2, end2)
#     print(area1, area2, high1_min, high2_min)
    if high2_min < high1_min and area2 > area1:
        return True
    return False

In [157]:
print(if_macd_back(603555, '20230109', 1))

False


In [161]:
# 遍历文件
import os

cur_date = '20230115'
dir_day = cur_date + '/data'
dir_60 = cur_date + '/data_60'
dir_30 = cur_date + '/data_30'
files_day  = os.listdir(dir_day)
files_60  = os.listdir(dir_60)
files_30  = os.listdir(dir_30)

num = 0
for f in files_day:
    stock_code = f.split('.')[0]
    data_day = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    data_60 = pd.read_csv(cur_date + '/data_60/' + str(stock_code) + '.csv')
    data_30 = pd.read_csv(cur_date + '/data_30/' + str(stock_code) + '.csv')
    macd_hist_day = data_day.loc[data_day.index[-1]]['macd_hist']
    macd_hist_60 = data_60.loc[data_60.index[-1]]['macd_hist']
    macd_hist_30 = data_30.loc[data_30.index[-1]]['macd_hist']
    flag_day = data_day.loc[data_day.index[-1]]['macd_hist'] > data_day.loc[data_day.index[-1]-1]['macd_hist']
    flag_60 = data_60.loc[data_60.index[-1]]['macd_hist'] > data_60.loc[data_60.index[-1]-1]['macd_hist']
    flag_30 = data_30.loc[data_30.index[-1]]['macd_hist'] > data_30.loc[data_30.index[-1]-1]['macd_hist']
    if macd_hist_60 < 0 and flag_60:
        print(stock_code, macd_hist_day)
        if if_macd_back(stock_code, cur_date, 60):
            print(stock_code, macd_hist_day, 'back')
        num += 1
print(num)


600048 0.0773574423051266
603555 -0.0189578014242223
600712 -0.0574242959747244
603569 -0.0026587745947248
600706 -0.009336456924329
601801 -0.0017816791257714
603025 -0.0965982207472288
603031 0.0454067047389361
600510 -0.0040422570808784
600262 0.0518485298784088
603192 -0.0129236447626278
601949 -0.0210429439234645
600316 0.0521939462687345
600855 0.0261495301108189
600128 0.0229992305644038
600101 0.0597952511305124
600129 0.3658310737356255
600897 -0.1192601629802244
600868 0.0070435785957498
605086 0.018369604036074
603839 -0.0891061215275007
603839 -0.0891061215275007 back
600505 0.0370728208447416
601155 0.0597310333972485
601800 -0.0438418525297541
600713 -0.0140424234108505
600908 0.026604471739355
600088 -0.0599185815942724
603230 0.0186699168909748
603032 0.2492998531894616
600249 -0.033190736358456
603998 0.0727932198261985
600261 0.0089333442967227
600498 0.0392778525026633
601989 0.0002085522826301
600467 -0.0142642168705064
600842 -0.0527591969753483
600117 0.0020779815

603060 -0.0926848846412327
603712 0.0879452598774698
600227 -0.0047070862237016
600582 0.0260862207871834
600743 -0.0191885437569001
600025 0.0345695737634309
600794 -0.0472142447660549
600780 0.0472987683509717
605188 -0.0957126664567598
600966 -0.0706436896750651
600033 -0.0063538970154633
600027 0.0703658363292959
600769 -0.0866967573634814
603506 -0.0099720816360266
600231 -0.0035948151335161
600225 0.1171078543795444
601113 0.0039731125114501
600386 0.0107905717115843
603843 -0.0195016683401376
600423 -0.0055830875822314
600351 -0.0196971846317947
603658 0.9032069034435318
600184 0.0461532005247792
600153 -0.0558547595222418
603466 0.0161597855945578
603315 0.2151099723005157
600807 0.0020229340664937
603103 -0.0464726564661963
603665 0.1882067370466053
600422 0.3221125763247303
600393 -0.0250259284924706
603856 0.1147313401629881
603739 0.0383860470266526
601890 0.0669105936663227
600556 0.0310815151769339
603261 0.1187571282419576
600768 0.0086147037550079
600998 -0.004446756968

In [196]:
# 计算macd面积
def if_macd_back_cur2(stock_code, date, time_span):
    data_dir = 'data'
    if time_span == 60:
        data_dir = 'data_60'
    elif time_span == 30:
        data_dir = 'data_30'
    file = os.path.join(date + '/' + data_dir, str(stock_code)+'.csv')
    data = pd.read_csv(file)
#     print(data)
#     print(data.loc[data.index[-1]])
    if data.loc[data.index[-1]]['macd_hist'] >= 0:
        return False
#     indexs = data[data['macd_hist'] < 0].index
    end2 = data.index[-1]
    if data.loc[end2]['Date'].find('2023') == -1:
        return False
    start2 = end2
    end1 = -1
    area2 = data.loc[end2]['macd_hist']
    area2_min = data.loc[end2]['Close']
    area2_max = data.loc[end2]['Close']
    for i in range(end2-1, -1, -1):
        if data.loc[i]['macd_hist'] > 0:
            start2 = i+1
            break
        else:
            area2 += data.loc[i]['macd_hist']
            if area2_min > data.loc[i]['Close']:
                area2_min = data.loc[i]['Close']
            if area2_max < data.loc[i]['Close']:
                area2_max = data.loc[i]['Close']
    if start2 == end2:
        return False
    end1_begin = start2-1
#     print(start2, end2)
    while True:
        for i in range(end1_begin,-1,-1):
            if data.loc[i]['macd_hist'] < 0:
                end1 = i
                break
        start1 = end1
        area1 = data.loc[end1]['macd_hist']
        area1_min = data.loc[end1]['Close']
        area1_max = data.loc[end1]['Close']
        for i in range(end1-1, -1, -1):
            if data.loc[i]['macd_hist'] > 0:
                start1 = i+1
                break
            else:
                area1 += data.loc[i]['macd_hist']
                if area1_min > data.loc[i]['Close']:
                    area1_min = data.loc[i]['Close']
                if area1_max < data.loc[i]['Close']:
                    area1_max = data.loc[i]['Close']
        if end1 <= 33:
            return False
        if end1 - start1 <= 1:
            end1_begin = end1-1
        else:
            break
    if start2 - end1 <= 4:
        return False
    area3_max = data.loc[end1+1]['Close']
    for i in range(end1+1, start2-1, 1):
        if area3_max < data.loc[i]['Close']:
            area3_max = data.loc[i]['Close']
#     print(start1, end1, start2, end2)
#     print(data.loc[start1]['Date'], data.loc[end1]['Date'], data.loc[start2]['Date'], data.loc[end2]['Date'])
#     print(area1, area1_min, area2, area2_min)
    if area1_min > area2_min and area1 < area2 and area1_max > area3_max:
        return True
    return False
    

In [199]:
# 遍历文件
import os

cur_date = '20230116'
dir_day = cur_date + '/data'
dir_60 = cur_date + '/data_60'
dir_30 = cur_date + '/data_30'
files_day  = os.listdir(dir_day)
# files_60  = os.listdir(dir_60)
# files_30  = os.listdir(dir_30)

for f in files_day:
    stock_code = f.split('.')[0]
    data_day = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
#     data_60 = pd.read_csv(cur_date + '/data_60/' + str(stock_code) + '.csv')
#     data_30 = pd.read_csv(cur_date + '/data_30/' + str(stock_code) + '.csv')
#     macd_hist_day = data_day.loc[data_day.index[-1]]['macd_hist']
#     macd_hist_60 = data_60.loc[data_60.index[-1]]['macd_hist']
#     macd_hist_30 = data_30.loc[data_30.index[-1]]['macd_hist']
#     flag_day = data_day.loc[data_day.index[-2]]['macd_hist'] > data_day.loc[data_day.index[-2]-1]['macd_hist']
#     flag_60 = data_60.loc[data_60.index[-1]]['macd_hist'] > data_60.loc[data_60.index[-1]-1]['macd_hist']
#     flag_30 = data_30.loc[data_30.index[-1]]['macd_hist'] > data_30.loc[data_30.index[-1]-1]['macd_hist']
    if data_day.loc[data_day.index[-3]]['macd_hist'] < 0 and data_day.loc[data_day.index[-4]]['macd_hist'] > data_day.loc[data_day.index[-3]]['macd_hist'] and data_day.loc[data_day.index[-3]]['macd_hist'] < data_day.loc[data_day.index[-2]]['macd_hist']:
#         print(stock_code)
        if if_macd_back_cur2(stock_code, cur_date, 1):
            print(stock_code, ' day back')


600117  day back
600158  day back
600491  day back


In [197]:
if_macd_back_cur2('603151', cur_date, 1)

False

In [187]:
data = pd.read_csv('20230116/data/603151.csv')

In [192]:
data[30:]

,Date,Open,Close,High,Low,hands,Volume,macd,macd_signal,macd_hist,ma10,ma30,rsi
30,2022-11-30,23.11,22.73,23.20,22.30,18353.0,41660948.0,NaN,NaN,NaN,23.805,21.395333,44.899735
31,2022-12-01,22.73,22.82,23.30,22.72,21221.0,48802546.0,NaN,NaN,NaN,23.557,21.406000,45.377516
32,2022-12-02,22.67,21.74,23.49,21.70,40827.0,91055103.0,NaN,NaN,NaN,23.339,21.437000,40.805005
33,2022-12-05,21.80,21.94,22.13,21.55,24054.0,52579580.0,0.815083,1.386828,-0.571745,23.043,21.483000,41.971140
34,2022-12-06,21.93,21.80,22.08,21.70,16435.0,35906420.0,0.684566,1.246375,-0.561809,22.759,21.588000,41.356961
35,2022-12-07,21.87,21.71,22.30,21.58,21386.0,46751606.0,0.567329,1.110566,-0.543237,22.529,21.704000,40.942183
36,2022-12-08,21.53,21.17,21.69,21.12,21064.0,44866082.0,0.425934,0.973640,-0.547706,22.264,21.798667,38.450440
37,2022-12-09,21.16,20.40,21.17,20.25,29932.0,61315858.0,0.248876,0.828687,-0.579811,22.023,21.909000,35.164092
38,2022-12-12,20.36,20.45,20.67,20.14,18016.0,36832806.0,0.111308,0.685211,-0.573903,21.738,21.996667,35.549308
39,2022-12-13,20.47,21.13,21.45,20.26,30834.0,64677974.0,0.056503,0.559469,-0.502966,21.589,22.047667,40.708745


In [165]:
def if_macd_back_cur(stock_code, date, time_span):
    data_dir = 'data'
    if time_span == 60:
        data_dir = 'data_60'
    elif time_span == 30:
        data_dir = 'data_30'
    file = os.path.join(date + '/' + data_dir, str(stock_code)+'.csv')
    data = pd.read_csv(file)
#     print(data)
#     print(data.loc[data.index[-1]])
    if data.loc[data.index[-1]]['macd_hist'] >= 0:
        return False
#     indexs = data[data['macd_hist'] < 0].index
    end2 = data.index[-1]
    if data.loc[end2]['Date'].find('2023') == -1:
        return False
    start2 = end2
    end1 = -1
    area2 = data.loc[end2]['macd_hist']
    area2_min = data.loc[end2]['Close']
    for i in range(end2-1, -1, -1):
        if data.loc[i]['macd_hist'] > 0:
            start2 = i+1
            break
        else:
            area2 += data.loc[i]['macd_hist']
            if area2_min > data.loc[i]['Close']:
                area2_min = data.loc[i]['Close']
    if start2 == end2:
        return False
    end1_begin = start2-1
    while True:
        for i in range(end1_begin,-1,-1):
            if data.loc[i]['macd_hist'] < 0:
                end1 = i
                break
        start1 = end1
        area1 = data.loc[end1]['macd_hist']
        area1_min = data.loc[end1]['Close']
        for i in range(end1-1, -1, -1):
            if data.loc[i]['macd_hist'] > 0:
                start1 = i+1
                break
            else:
                area1 += data.loc[i]['macd_hist']
                if area1_min > data.loc[i]['Close']:
                    area1_min = data.loc[i]['Close']
        if end1 - start1 <= 1:
            end1_begin = end1-1
        else:
            break
    print(start1, end1, start2, end2)
    print(data.loc[start1]['Date'], data.loc[end1]['Date'], data.loc[start2]['Date'], data.loc[end2]['Date'])
    print(area1, area1_min, area2, area2_min)
    if area1_min > area2_min and area1 < area2:
        return True
    return False

In [164]:
# 遍历文件
import os

cur_date = '20230110'
dir_day = cur_date + '/data'
dir_60 = cur_date + '/data_60'
dir_30 = cur_date + '/data_30'
files_day  = os.listdir(dir_day)
files_60  = os.listdir(dir_60)
files_30  = os.listdir(dir_30)

num = 0
for f in files_day:
    stock_code = f.split('.')[0]
    data_day = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    data_60 = pd.read_csv(cur_date + '/data_60/' + str(stock_code) + '.csv')
    data_30 = pd.read_csv(cur_date + '/data_30/' + str(stock_code) + '.csv')
    if if_macd_back_cur(stock_code, cur_date, 1):
        print(stock_code, 'day')


600712 day
600302 day
600854 day
600077 day
600728 day
600981 day
600361 day
689009 day
600834 day
601086 day
601898 day
603136 day
600619 day
601918 day
603538 day
603229 day
600085 day


In [167]:
if_macd_back_cur('600077', cur_date, 1)

277 295 298 299
2022-12-08 2023-01-04 2023-01-09 2023-01-10
-0.4478795222399599 2.63 -0.0057506527544213 2.59


True